# Running GET-DEM

This notebook allows you to do the following within the ADE:

1. Run GET-DEM locally (i.e. _not_ via the DPS).
1. Register the GET-DEM algorithm defined in this repository.
1. Submit jobs to the DPS to run the registered algorithm.

In order to use this notebook within the ADE, you must first create the `dem`
`conda` environment, with the `--dev` option to install development
dependencies:

```plain
./build-env.sh --dev
```

Once the `dem` `conda` environment is created with development dependencies, you
must then install it as an `ipython kernel`:

```plain
conda run -n dem ipython kernel install --user --name dem
```

Finally, to run the cells in this notebook, you must choose the `dem` kernel.

## Choose Bounding Box

The bounding box should be a single string of four lat/lon, in the order:
[left  bottom  right top]

In [ ]:
bbox_for = {
    # Mt Wilson (near JPL) (small, ~5 sec)
    "MtWilson": (-118.06817, 34.22169, -118.05801, 34.22822),
    # uses ~5-6 GB RAM, takes a few minutes to run
    "WesternUS-CA-NV": (-124.81360, 32.44506, -113.75989, 42.24498),
    # very large bbox options -- might run out of memory
    "WesternUS-CA-WA-Chicago-NewOrleans": (-125.53514, 25.42150, -87.06960, 49.22105),
    "WesternUS-CA-CO": (-125.41615, 31.03621, -101.80916, 49.17324),
    "Italy": (6.26868, 36.00380, 18.57179, 47.28139),  # requires more than 8 GB RAM
}

def bbox_arg_for(name: str) -> str:
    left, bottom, right, top = bbox_for[name]
    return f"{left} {bottom} {right} {top}"

## Run GET-DEM within the ADE

If you want to test GET-DEM locally, before registering the algorithm or
submitting jobs to run in the DPS, you can simply run the following, once you
have assigned a value to `bbox` above:

In [ ]:
# IMPORTANT: The run.sh script expects EXACTLY 3 arguments: bbox, compute, and
# scalene_args.  This is because the DPS will always call the run.sh script with
# exactly 3 arguments, as defined in algorithm_config.yml.  This is true even if
# some inputs are not supplied during job submission.

# THEREFORE, if you want to supply only the bbox, you must still supply the
# other arguments as empty strings (empty quotes).  The run.sh script will
# handle this correctly.  ALSO, the 4 values of the bbox must be separated by
# spaces, not commas, and the entire string must be enclosed in quotes.

# Do a test run via CLI to ensure input arguments are passed correctly to the algorithm.
!./run.sh "{bbox_arg_for('MtWilson')}" "true" "--reduced-profile --html --outfile output/profile.html"

## Initialize MAAP

You must create a `MAAP` instance in order to either regsiter the algorithm or
submit jobs.

In [ ]:
from maap.maap import MAAP

import get_dem

maap = MAAP(maap_host="api.maap-project.org")
username = maap.profile.account_info()["username"]
username

## Register the Algorithm

Whenever you modify the code in `get_dem.py` or the algorithm configuration in
`algorithm_config.yml`, you must register the algorithm in order to run the
new code as a DPS job.

However, before doing so, you should update the following places where the
algorithm's version appears:

- `algorithm_version` within `algorithm_config.yml`
- `__version__` within `get_dem.py`

**NOTE:** You should change both to the _same_ value.

**NOTE:** If you attempt to re-register a version of the algorithm that has
already been registered, you will receive a "500" error.  Although it is
possible for a "500" error to indicate some other problem with the server, in
this case, it likely means that the version of the algorithm is already
registered.  It's a known issue that the server should instead return an
appropriate message indicating the problem, rather than a general "500" error
without any details.

In [ ]:
# Register Algorithm from YAML file

# r = maap.register_algorithm_from_yaml_file("algorithm_config.yml")
# r.raise_for_status()
# r.json()

## Submit Job(s)

In [ ]:
version = get_dem.__version__
bbox_name = "Italy"
# True to perform compute-intensive work
compute = True

# Number of DPS jobs to launch
n_jobs = 1
queues = ["maap-dps-worker-32vcpu-64gb"]

# Submit DPS Jobs
jobs = [
    maap.submitJob(
        algo_id="get-dem",  # name of the registered algorithm
        version=version,
        username=username,
        queue=queue,
        identifier=f"{bbox_name}__{'' if compute else 'no-'}compute__{queue}",  # job tag
        # for benchmarking consistency, use same bbox for all jobs
        bbox=bbox_arg_for(bbox_name),
        compute="true" if compute else "",
    )
    for _ in range(n_jobs)
    for queue in queues
]

## Check Job Status(es)

In [ ]:
# Monitor the job status via JN.
# For a GUI, go to "View and Submit Jobs" via the Launcher
n_failed = 0
n_succeeded = 0
n_running = 0
n_except = 0
n_accepted = 0

for j in jobs:
    try:
        j.retrieve_attributes()
    except:
        n_except += 1
        continue
    if j.status.lower() == 'failed':
        n_failed += 1
    elif j.status.lower() == 'succeeded':
        n_succeeded += 1
    elif j.status.lower() == 'running':
        n_running += 1
    elif j.status.lower() == 'accepted':
        n_accepted += 1
    else:
        print(j.status)

print("failed: ", n_failed)
print("accepted: ", n_accepted)
print("running: ", n_running)
print("succeeded: ", n_succeeded)
print("except: ", n_except)

In [ ]:
print(type(jobs[0]))
print(jobs[0])